In [1]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter as sg_filter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, input_data_split
from wattile.time_processing import add_processed_time_columns
from wattile.models.liangs_model import main as liangs_model
PROJECT_DIRECTORY = Path().resolve().parent.parent

In [2]:
#########################################################################################
# building the S2S model
class S2S_Model(nn.Module):
    def __init__(self, cell_type, input_size, hidden_size, use_cuda):
        super(S2S_Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.cell_type = cell_type

        if self.cell_type not in ['rnn', 'gru', 'lstm']:
            raise ValueError(self.cell_type, " is not an appropriate cell type. Please select one of rnn, gru, or lstm.")
        if self.cell_type == 'rnn':
            self.Ecell = nn.RNNCell(self.input_size, self.hidden_size)
            self.Dcell = nn.RNNCell(1, self.hidden_size)
        if self.cell_type == 'gru':
            self.Ecell = nn.GRUCell(self.input_size, self.hidden_size)
            self.Dcell = nn.GRUCell(1, self.hidden_size)
        if self.cell_type == 'lstm':
            self.Ecell = nn.LSTMCell(self.input_size, self.hidden_size)
            self.Dcell = nn.LSTMCell(1, self.hidden_size)

        self.lin_usage = nn.Linear(self.hidden_size, 1)
        self.use_cuda = use_cuda
        self.init()

    # function to intialize weight parameters. Refer to Saxe at al. paper that explains why to use orthogonal init weights
    def init(self):
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            for p in self.parameters():
                if p.dim() > 1:
                    init.orthogonal_(p.data, gain=1.0)
                if p.dim() == 1:
                    init.constant_(p.data, 0.0)
        elif self.cell_type == 'lstm':
            for p in self.parameters():
                if p.dim() > 1:
                    init.orthogonal_(p.data, gain=1.0)
                if p.dim() == 1:
                    init.constant_(p.data, 0.0)
                    init.constant_(p.data[self.hidden_size:2*self.hidden_size], 1.0)

    def consume(self, x):
        # encoder forward function
        # for rnn and gru
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            h = torch.zeros(x.shape[0], self.hidden_size)
            if self.use_cuda:
                h = h.cuda()
            for T in range(x.shape[1]):
                h = self.Ecell(x[:, T, :], h)
            pred_usage = self.lin_usage(h)
        # for lstm
        elif self.cell_type == 'lstm':
            h0 = torch.zeros(x.shape[0], self.hidden_size)
            c0 = torch.zeros(x.shape[0], self.hidden_size)
            if self.use_cuda:
                h0 = h0.cuda()
                c0 = c0.cuda()
            h = (h0, c0)
            for T in range(x.shape[1]):
                h = self.Ecell(x[:, T, :], h)
            pred_usage = self.lin_usage(h[0])
        return pred_usage, h

    def predict(self, pred_usage, h, target_length):
        # decoder forward function
        preds = []
        # for rnn and gru
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            for step in range(target_length):
                h = self.Dcell(pred_usage, h)
                pred_usage = self.lin_usage(h)
                preds.append(pred_usage.unsqueeze(1))
            preds = torch.cat(preds, 1)
        # for lstm
        elif self.cell_type == 'lstm':
            for step in range(target_length):
                h = self.Dcell(pred_usage, h)
                pred_usage = self.lin_usage(h[0])
                preds.append(pred_usage.unsqueeze(1))
            preds = torch.cat(preds, 1)
        return preds

#########################################################################################
# Bahdanau Attention model
# refer to : AuCson github code
# building the model
class S2S_BA_Model(nn.Module):
    def __init__(self, cell_type, input_size, hidden_size, use_cuda):
        super(S2S_BA_Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.use_cuda = use_cuda
        self.cell_type = cell_type

        if self.cell_type not in ['rnn', 'gru', 'lstm']:
            raise ValueError(self.cell_type, " is not an appropriate cell type. Please select one of rnn, gru, or lstm.")
        if self.cell_type == 'rnn':
            self.Ecell = nn.RNNCell(self.input_size, self.hidden_size)
            self.Dcell = nn.RNNCell(1+self.hidden_size, self.hidden_size)
        if self.cell_type == 'gru':
            self.Ecell = nn.GRUCell(self.input_size, self.hidden_size)
            self.Dcell = nn.GRUCell(1+self.hidden_size, self.hidden_size)
        if self.cell_type == 'lstm':
            self.Ecell = nn.LSTMCell(self.input_size, self.hidden_size)
            self.Dcell = nn.LSTMCell(1+self.hidden_size, self.hidden_size)

        self.Wattn_energies = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.Wusage = nn.Linear(self.hidden_size, 1)
        self.Wout = nn.Linear(1+self.hidden_size*2, self.hidden_size)
        self.v = nn.Parameter(torch.rand(self.hidden_size))
        stdv = 1./math.sqrt(self.v.size(0))
        self.v.data.normal_(mean=0, std=stdv)
        self.init()

# function to intialize weight parameters
    def init(self):
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            for p in self.parameters():
                if p.dim() > 1:
                    init.orthogonal_(p.data, gain=1.0)
                if p.dim() == 1:
                    init.constant_(p.data, 0.0)
        elif self.cell_type == 'lstm':
            for p in self.parameters():
                if p.dim() > 1:
                    init.orthogonal_(p.data, gain=1.0)
                if p.dim() == 1:
                    init.constant_(p.data, 0.0)
                    init.constant_(p.data[self.hidden_size:2*self.hidden_size], 1.0)

    def consume(self, x):
        # for rnn and gru
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            # encoder forward function
            h = torch.zeros(x.shape[0], self.hidden_size)
            encoder_outputs = torch.zeros(x.shape[1], x.shape[0], self.hidden_size)
            if self.use_cuda:
                h = h.cuda()
                encoder_outputs = encoder_outputs.cuda()
            # encoder part
            for T in range(x.shape[1]):
                h = self.Ecell(x[:, T, :], h)
                encoder_outputs[T] = h
            pred_usage = self.Wusage(h)
        # for lstm
        elif self.cell_type == 'lstm':
            h0 = torch.zeros(x.shape[0], self.hidden_size)
            c0 = torch.zeros(x.shape[0], self.hidden_size)
            encoder_outputs = torch.zeros(x.shape[1], x.shape[0], self.hidden_size)
            if self.use_cuda:
                h0 = h0.cuda()
                c0 = c0.cuda()
                encoder_outputs = encoder_outputs.cuda()
            h = (h0, c0)
            for T in range(x.shape[1]):
                h = self.Ecell(x[:, T, :], h)
                encoder_outputs[T] = h[0]
            pred_usage = self.Wusage(h[0])
        return pred_usage, h, encoder_outputs

    def predict(self, pred_usage, h, encoder_outputs, target_length):
        # decoder with attention function
        preds = []
        # for rnn and gru
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            for step in range(target_length):
                h_copies = h.expand(encoder_outputs.shape[0], -1, -1)
                energies = torch.tanh(self.Wattn_energies(torch.cat((h_copies, encoder_outputs), 2)))
                score = torch.sum(self.v * energies, dim=2)
                attn_weights = score.t()
                attn_weights = torch.softmax(attn_weights, dim=1).unsqueeze(1)
                context = attn_weights.bmm(encoder_outputs.transpose(0,1)).squeeze(1)
                gru_input = torch.cat((pred_usage, context), 1)
                h = self.Dcell(gru_input, h)
                output = self.Wout(torch.cat((pred_usage, h, context), 1))
                pred_usage = self.Wusage(output)
                preds.append(pred_usage.unsqueeze(1))
            preds = torch.cat(preds, 1)
        # for lstm
        elif self.cell_type == 'lstm':
            for step in range(target_length):
                h_copies = h[0].expand(encoder_outputs.shape[0], -1, -1)
                energies = torch.tanh(self.Wattn_energies(torch.cat((h_copies, encoder_outputs), 2)))
                score = torch.sum(self.v * energies, dim=2)
                attn_weights = score.t()
                attn_weights = torch.softmax(attn_weights, dim=1).unsqueeze(1)
                context = attn_weights.bmm(encoder_outputs.transpose(0,1)).squeeze(1)
                gru_input = torch.cat((pred_usage, context), 1)
                h = self.Dcell(gru_input, h)
                output = self.Wout(torch.cat((pred_usage, h[0], context), 1))
                pred_usage = self.Wusage(output)
                preds.append(pred_usage.unsqueeze(1))
            preds = torch.cat(preds, 1)
        return preds

#############################################################################################3
# Luong Attention module
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, " is not an appropriate attention method, please select one of dot, general, or concat.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, self.hidden_size)
        if self.method == 'concat':
            self.attn = nn.Linear(2*self.hidden_size, self.hidden_size)
            self.v = nn.Parameter(torch.rand(self.hidden_size))
            stdv = 1./math.sqrt(self.v.size(0))
            self.v.data.normal_(mean=0, std=stdv)

    def dot_score(self, hidden, encoder_output):
        attn_energies = torch.sum(hidden*encoder_output, dim=2)
        return attn_energies

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        attn_energies = torch.sum(hidden*energy, dim=2)
        return attn_energies

    def concat_score(self, hidden, encoder_output):
        energy = torch.tanh(self.attn(torch.cat((hidden.expand(encoder_output.shape[0], -1, -1),
                            encoder_output), 2)))
        return torch.sum(self.v * energy, dim=2)

    # calculate the attention weights (energies) based on the given method
    def forward(self, hidden, encoder_outputs):
        if self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
        elif self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        attn_energies = attn_energies.t()
        attn_weights = torch.softmax(attn_energies, dim=1).unsqueeze(1)
        return attn_weights

#########################################################################################
#  building the S2S LA model
class S2S_LA_Model(nn.Module):
    def __init__(self, cell_type, attn_method, input_size, hidden_size, use_cuda):
        super(S2S_LA_Model, self).__init__()
        self.cell_type = cell_type
        self.attn_method = attn_method
        self.input_size = input_size
        self.hidden_size = hidden_size

        if self.cell_type == 'rnn':
            self.Ecell = nn.RNNCell(self.input_size, self.hidden_size)
            self.Dcell = nn.RNNCell(1, self.hidden_size)
        if self.cell_type == 'gru':
            self.Ecell = nn.GRUCell(self.input_size, self.hidden_size)
            self.Dcell = nn.GRUCell(1, self.hidden_size)
        if self.cell_type == 'lstm':
            self.Ecell = nn.LSTMCell(self.input_size, self.hidden_size)
            self.Dcell = nn.LSTMCell(1, self.hidden_size)

        self.lin_usage = nn.Linear(hidden_size, 1)
        self.lin_concat = nn.Linear(hidden_size*2, hidden_size)
        self.attn = Attn(self.attn_method, self.hidden_size)
        self.use_cuda = use_cuda
        self.init()

    # function to intialize weight parameters
    def init(self):
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            for p in self.parameters():
                if p.dim() > 1:
                    init.orthogonal_(p.data, gain=1.0)
                if p.dim() == 1:
                    init.constant_(p.data, 0.0)
        elif self.cell_type == 'lstm':
            for p in self.parameters():
                if p.dim() > 1:
                    init.orthogonal_(p.data, gain=1.0)
                if p.dim() == 1:
                    init.constant_(p.data, 0.0)
                    init.constant_(p.data[self.hidden_size:2*self.hidden_size], 1.0)

    def consume(self, x):
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            # encoder forward function
            h = torch.zeros(x.shape[0], self.hidden_size)
            encoder_outputs = torch.zeros(x.shape[1], x.shape[0], self.hidden_size)
            if self.use_cuda:
                h = h.cuda()
                encoder_outputs = encoder_outputs.cuda()
            # encoder part
            for T in range(x.shape[1]):
                h = self.Ecell(x[:, T, :], h)
                encoder_outputs[T] = h
            pred_usage = self.lin_usage(h)
        elif self.cell_type == 'lstm':
            h0 = torch.zeros(x.shape[0], self.hidden_size)
            c0 = torch.zeros(x.shape[0], self.hidden_size)
            encoder_outputs = torch.zeros(x.shape[1], x.shape[0], self.hidden_size)
            if self.use_cuda:
                h0 = h0.cuda()
                c0 = c0.cuda()
                encoder_outputs = encoder_outputs.cuda()
            h = (h0, c0)
            for T in range(x.shape[1]):
                h = self.Ecell(x[:, T, :], h)
                encoder_outputs[T] = h[0]
            pred_usage = self.lin_usage(h[0])
        return pred_usage, h, encoder_outputs

    def predict(self, pred_usage, h, encoder_outputs, target_length):
        # decoder with attention function
        preds = []
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            for step in range(target_length):
                h = self.Dcell(pred_usage, h)
                attn_weights = self.attn(h, encoder_outputs)
                context = attn_weights.bmm(encoder_outputs.transpose(0,1))
                context = context.squeeze(1)
                concat_input = torch.cat((h, context), 1)
                concat_output = torch.tanh(self.lin_concat(concat_input))
                pred_usage = self.lin_usage(concat_output)
                preds.append(pred_usage.unsqueeze(1))
            preds = torch.cat(preds, 1)
        elif self.cell_type == 'lstm':
            for step in range(target_length):
                h = self.Dcell(pred_usage, h)
                attn_weights = self.attn(h[0], encoder_outputs)
                context = attn_weights.bmm(encoder_outputs.transpose(0,1))
                context = context.squeeze(1)
                concat_input = torch.cat((h[0], context), 1)
                concat_output = torch.tanh(self.lin_concat(concat_input))
                pred_usage = self.lin_usage(concat_output)
                preds.append(pred_usage.unsqueeze(1))
            preds = torch.cat(preds, 1)
        return preds
    
# loss function, qs here is an integer, not a list of integer
def quantile_loss(output, target, qs, window_target_size):
    """
    Computes loss for quantile methods.
    :param output: (Tensor)
    :param target: (Tensor)
    :param qs: (int)
    :param window_target_size: (int)
    :return: (Tensor) Loss for this study (single number)
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    resid = target - output
    tau = torch.tensor([qs], device=device).repeat_interleave(window_target_size)

    alpha = 0.001
    log_term = torch.zeros_like(resid, device=device)
    log_term[resid < 0] = torch.log(1 + torch.exp(resid[resid < 0] / alpha)) - (
        resid[resid < 0] / alpha
    )
    log_term[resid >= 0] = torch.log(1 + torch.exp(-resid[resid >= 0] / alpha))
    loss = resid * tau + alpha * log_term
    loss = torch.mean(torch.mean(loss, 0))

    return loss

# A key function to convert from 2D data to 3D data. 
def generate_windows(train_source, test_source, WINDOW_SOURCE_SIZE, WINDOW_TARGET_SIZE):
    x_train = []
    y_usage_train = []
    x_test = []
    y_usage_test = []

    # for training data
    # idxs = np.random.choice(train_source.shape[0]-(WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE), train_source.shape[0]-(WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE), replace=False)
    idxs = np.arange(0, len(train_source)-(WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE))

    for idx in idxs:
        x_train.append(train_source[idx:idx+WINDOW_SOURCE_SIZE].reshape((1, WINDOW_SOURCE_SIZE, train_source.shape[1])) )
        y_usage_train.append(train_source[idx+WINDOW_SOURCE_SIZE:idx+WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE, -1].reshape((1, WINDOW_TARGET_SIZE, 1)) )

    x_train = np.concatenate(x_train, axis=0) # make them arrays and not lists
    y_usage_train = np.concatenate(y_usage_train, axis=0)

    # for testing data
    idxs = np.arange(0, len(test_source)-(WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE), WINDOW_TARGET_SIZE)

    for idx in idxs:
        x_test.append(test_source[idx:idx+WINDOW_SOURCE_SIZE].reshape((1, WINDOW_SOURCE_SIZE, test_source.shape[1])) )
        y_usage_test.append(test_source[idx+WINDOW_SOURCE_SIZE:idx+WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE, -1].reshape((1, WINDOW_TARGET_SIZE, 1)) )

    x_test = np.concatenate(x_test, axis=0) # make them arrays and not lists
    y_usage_test = np.concatenate(y_usage_test, axis=0)

    return x_train, y_usage_train, x_test, y_usage_test

### reading configs

In [3]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# # main configs file
# with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################
# code testing configs file
with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
    configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["exp_dir"] = str(exp_dir)
configs["num_epochs"] = 10
configs["data_dir"] = str(PROJECT_DIRECTORY / "tests" / "data")

configs

{'building': 'Synthetic Site',
 'target_var': 'Synthetic Site Electricity Main Total Power',
 'start_time': '2018-01-01T00:00:00-07:00',
 'end_time': '2022-01-01T00:00:00-07:00',
 'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
  'Synthetic Weather Station Diffuse Horizontal Irradiance',
  'Synthetic Weather Station Direct Normal Irradiance',
  'Synthetic Weather Station Dry Bulb Temperature',
  'Synthetic Weather Station Global Horizontal Irradiance',
  'Synthetic Weather Station Relative Humidity',
  'Synthetic Weather Station Wind Speed'],
 'arch_version': 4,
 'exp_id': 'Debugging_for_rolling_stats',
 'arch_type': 'RNN',
 'arch_type_variant': 'lstm',
 'transformation_method': 'minmaxscale',
 'train_batch_size': 5,
 'val_batch_size': 1,
 'convert_csvs': False,
 'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir',
 'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\tests\\data'

### override config if necessary

In [4]:
# configs["feat_stats"]["window_width"] = '15min'
# configs["feat_stats"]["window_increment"] = '15min'

### reading data

In [5]:
site = configs['building']

In [6]:
data = read_dataset_from_file(configs)
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


### feature extraction

In [7]:
# assert we have the correct columns and order them
data = correct_predictor_columns(configs, data)

# sort and trim data specified time period
data = correct_timestamps(configs, data)

# Add time-based features
data = add_processed_time_columns(data, configs)

# Add statistics features
data = resample_or_rolling_stats(data, configs)

data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,DOW_binary_reg_0_min,...,DOW_binary_reg_0_mean,DOW_binary_reg_1_mean,DOW_binary_reg_2_mean,DOW_binary_reg_3_mean,DOW_binary_reg_4_mean,DOW_binary_reg_5_mean,DOW_binary_reg_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,103.650002
2021-12-01 07:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:45:00+00:00,16.614972,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.349998
2021-12-01 08:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,102.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.000000,0.998135,-1.608123e-16,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.000000,0.997859,-6.540313e-02,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,102.709602
2021-12-08 06:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.000000,0.991445,-1.305262e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,105.972198


In [8]:
data_wTS = data.copy()

# =======================================
# original workflow
# =======================================

In [9]:
#################################################################################################################################################
# main function
def main(seed, cuda, cell_type, attention_model, la_method, window_source_size,
            window_target_size, epochs, batch_size, hs, save_model, loss_function_qs, normalization):
    t0 = time.time()
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    dataset = data_wTS.astype(np.float32).copy()
        
    usage_actual = dataset[configs["target_var"]]
    mu_usage = dataset[configs["target_var"]].mean()
    std_usage = dataset[configs["target_var"]].std()
    dataset = dataset.values

    # Normalization
    if normalization: 
        print(f"Transforming data to 0 mean and unit var")
        MU = dataset.mean(0) # 0 means take the mean of the column
        dataset = dataset - MU
        STD = dataset.std(0) # same with std here
        dataset = dataset / STD
    else:
        MU = dataset.mean(0) # 0 means take the mean of the column
        MU = 0
        dataset = dataset - MU
        STD = dataset.std(0) # same with std here
        STD = 1
        dataset = dataset / STD

    print("Generating training and test data...")
    WINDOW_SOURCE_SIZE = window_source_size
    WINDOW_TARGET_SIZE = window_target_size

    # getting actual usage vector, aligning with predicted values vector. Aka remove first window_source_size and remaining
    usage_actual = usage_actual.values
    usage_actual = usage_actual[int(dataset.shape[0]*0.80):]
    usage_actual = usage_actual[WINDOW_SOURCE_SIZE:]

    # training testing split: 80% train, 20% test
    train_source = dataset[:int(dataset.shape[0]*0.80)]
    test_source = dataset[int(dataset.shape[0]*0.80):]

    X_train, Y_train_usage, X_test, Y_test_usage = generate_windows(train_source, test_source, window_source_size, window_target_size)
    print("Created {} train samples and {} test samples".format(X_train.shape[0], X_test.shape[0]))
    idxs = np.arange(0, len(test_source)-(WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE), WINDOW_TARGET_SIZE)
    remainder = len(test_source) - (idxs[-1] + WINDOW_SOURCE_SIZE+WINDOW_TARGET_SIZE)
    usage_actual = usage_actual[:-remainder]

#################################################################################################################################################
# call the model
    #print("Creating model...")
    INPUT_SIZE = X_train.shape[-1]
    HIDDEN_SIZE = hs
    CELL_TYPE = cell_type
    LA_METHOD = la_method

    # call the respective model
    if attention_model == 'none':
        model = S2S_Model(CELL_TYPE, INPUT_SIZE, HIDDEN_SIZE, use_cuda=cuda)

    elif attention_model == 'BA':
        model = S2S_BA_Model(CELL_TYPE, INPUT_SIZE, HIDDEN_SIZE, use_cuda=cuda)

    elif attention_model == 'LA':
        model = S2S_LA_Model(CELL_TYPE, LA_METHOD, INPUT_SIZE, HIDDEN_SIZE, use_cuda=cuda)

    if cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        model.cuda()

    print("MODEL ARCHITECTURE IS: ")
    print(model)

    print("\nModel parameters are on cuda: {}".format(next(model.parameters()).is_cuda))

    opt = optim.Adam(model.parameters(), lr=1e-3)
#     loss_fn = nn.MSELoss(reduction='sum')
    loss_fn = quantile_loss
    
    EPOCHES = epochs
    BATCH_SIZE = batch_size

    print("\nStarting training...")

    train_loss = []
    test_loss = []

    for epoch in range(EPOCHES):
        t_one_epoch = time.time()
        print("Epoch {}".format(epoch+1))
        total_usage_loss = 0
        for b_idx in range(0, X_train.shape[0], BATCH_SIZE):

            x = torch.from_numpy(X_train[b_idx:b_idx+BATCH_SIZE]).float()
            y_usage = torch.from_numpy(Y_train_usage[b_idx:b_idx+BATCH_SIZE]).float()

            if cuda:
                x = x.cuda()
                y_usage = y_usage.cuda()

            # encoder forward, for respective models (with and without attention)
            if attention_model == 'none':
                pred_usage, h = model.consume(x)

            elif attention_model == 'BA':
                pred_usage, h, encoder_outputs = model.consume(x)

            elif attention_model == 'LA':
                pred_usage, h, encoder_outputs = model.consume(x)

            # decoder forward, for respective models
            if attention_model == 'none':
                preds = model.predict(pred_usage, h, WINDOW_TARGET_SIZE)

            elif attention_model == 'BA':
                preds = model.predict(pred_usage, h, encoder_outputs, WINDOW_TARGET_SIZE)

            elif attention_model == 'LA':
                preds = model.predict(pred_usage, h, encoder_outputs, WINDOW_TARGET_SIZE)

            # compute lose
            loss_usage = loss_fn(preds, y_usage, qs = loss_function_qs, window_target_size=window_target_size)

            # backprop and update
            opt.zero_grad()

            loss_usage.sum().backward()

            opt.step()

#             total_usage_loss += loss_usage.item()
            
        train_loss.append(total_usage_loss)
        print("\tTRAINING: {} total train USAGE loss.\n".format(total_usage_loss))

#################################################################################################################################################
# TESTING
        y_usage = None
        pred_usage = None
        preds = None
        total_usage_loss = 0
        all_preds = []

        for b_idx in range(0, X_test.shape[0], BATCH_SIZE):
            with torch.no_grad():
                x = torch.from_numpy(X_test[b_idx:b_idx+BATCH_SIZE])
                y_usage = torch.from_numpy(Y_test_usage[b_idx:b_idx+BATCH_SIZE])

                if cuda:
                    x = x.cuda()
                    y_usage = y_usage.cuda()

                # encoder forward, for respective models
                if attention_model == 'none':
                    pred_usage, h = model.consume(x)

                elif attention_model == 'BA':
                    pred_usage, h, encoder_outputs = model.consume(x)

                elif attention_model == 'LA':
                    pred_usage, h, encoder_outputs = model.consume(x)

                # decoder forward, for respective models
                if attention_model == 'none':
                    preds = model.predict(pred_usage, h, WINDOW_TARGET_SIZE)

                elif attention_model == 'BA':
                    preds = model.predict(pred_usage, h, encoder_outputs, WINDOW_TARGET_SIZE)

                elif attention_model == 'LA':
                    preds = model.predict(pred_usage, h, encoder_outputs, WINDOW_TARGET_SIZE)

                # compute loss
                loss_usage = loss_fn(preds, y_usage, qs = loss_function_qs, window_target_size=window_target_size)

                if (epoch == epochs-1):
                    all_preds.append(preds)

        test_loss.append(total_usage_loss)

        print("\tTESTING: {} total test USAGE loss".format(total_usage_loss))
        print("\tTESTING:\n")
        print("\tSample of prediction:")
        print("\t\t TARGET: {}".format(y_usage[-1].cpu().detach().numpy().flatten()))
        print("\t\t   PRED: {}\n\n".format(preds[-1].cpu().detach().numpy().flatten()))

        y_last_usage = y_usage[-1].cpu().detach().numpy().flatten()
        pred_last_usage = preds[-1].cpu().detach().numpy().flatten()
        t2_one_epoch = time.time()
        time_one_epoch = t2_one_epoch - t_one_epoch
        print("TIME OF ONE EPOCH: {} seconds and {} minutes".format(time_one_epoch, time_one_epoch/60.0))

####################################################################################################
# SAVING MODEL
    if save_model:
        torch.save(model.state_dict(), "MODEL_w:__seed={}_cell_type={}_attention_model={}_la_method={}_T={}_N={}_bs={}_hs={}".format(
            seed, cell_type, attention_model, la_method,
            window_source_size, window_target_size, batch_size, hs))

#################################################################################################################################################
# RESULTS
    # for plotting and accuracy
    preds = torch.cat(all_preds, 0)
    preds = preds.cpu().detach().numpy().flatten()
    actual = Y_test_usage.flatten()

    # for loss plotting
    train_loss_array = np.asarray(train_loss)
    test_loss_array = np.asarray(test_loss)
    len_loss = np.arange(len(train_loss_array))

    # unnormalizing 1
    if normalization:
        preds_unnorm = (preds*std_usage) + mu_usage
    else:
        preds_unnorm = preds

    # using the actual usage from top of script here
    mae3 = (sum(abs(usage_actual - preds_unnorm)))/(len(usage_actual))
    mape3 = (sum(abs((usage_actual - preds_unnorm)/usage_actual)))/(len(usage_actual))

    # for std
    mape_s = (abs((usage_actual - preds_unnorm)/usage_actual))
    s = mape_s.std()
    mae_s = abs(usage_actual - preds_unnorm)
    s2 = mae_s.std()
    print("\n\tACTUAL ACC. RESULTS: MAE, MAPE: {} and {}%".format(mae3, mape3*100.0))
    
    if not os.path.exists(f'results/{site}'):
        os.makedirs(f'results/{site}')
    
    pd.DataFrame(preds, columns = [f'q{loss_function_qs}']).to_csv(f'results/{site}/q{loss_function_qs}.csv', index = None)
    pd.DataFrame(actual, columns = [f'actual']).to_csv(f'results/{site}/actual.csv', index = None)

    # total time of run
    t1 = time.time()
    total = t1-t0
    print("\nTIME ELAPSED: {} seconds OR {} minutes".format(total, total/60.0))
    print("\nEnd of run")
    plt.show()
    for_plotting = [usage_actual, preds_unnorm, y_last_usage, pred_last_usage]
    return s, s2, mape_s, mae_s, mae3, mape3, total/60.0, train_loss, test_loss, for_plotting, MU, STD

In [10]:
# # cafe data is too large to set a high epochs number to run on my local machine
# n_epochs = 100

# for loss_function_qs in [0.05, 0.25, 0.50, 0.75, 0.95]: #[0.50]:#[0.05, 0.50, 0.95]:
#     print(f"Processing loss function alpha: {loss_function_qs}")
#     if __name__ == "__main__":
#         s, s2, mape_s, mae_s, mae, mape, total_mins, train_loss, test_loss, for_plotting, MU, STD = main(seed=42, cuda=False,
#             cell_type='lstm', attention_model='BA', la_method='none',
#             window_source_size=12, window_target_size=3, epochs=n_epochs,
#             batch_size=256, hs=64, save_model=False, loss_function_qs = loss_function_qs, normalization = False)
        
# site = "synthetic"
# q95 = pd.read_csv(f'results/{site}/q0.95.csv')
# q75 = pd.read_csv(f'results/{site}/q0.75.csv')
# q50 = pd.read_csv(f'results/{site}/q0.5.csv')
# q25 = pd.read_csv(f'results/{site}/q0.25.csv')
# q05 = pd.read_csv(f'results/{site}/q0.05.csv')
# actual = pd.read_csv(f'results/{site}/actual.csv')

# plt.figure(figsize=(15, 4.5))

# plt.title(f'{site} Whole Building Real Power Total Prediction')

# plt.plot(actual, label = 'actual')
# plt.plot(sg_filter(q95.values.flatten(),5,2), label = 'Q95', color = '#a2a2a2')
# plt.plot(sg_filter(q75.values.flatten(),5,2), label = 'Q75', color = '#5b5b5b')
# plt.plot(sg_filter(q50.values.flatten(),5,2), label = 'Medium', color = '#322b2b')
# plt.plot(sg_filter(q25.values.flatten(),5,2), label = 'Q25', color = '#5b5b5b')
# plt.plot(sg_filter(q05.values.flatten(),5,2), label = 'Q5', color = '#a2a2a2')

# plt.legend()
# # plt.ylim(80,140)
# n = 0
# # plt.xlim(240 + n, 480 + n)

# plt.ylabel(f'{site} Whole Building Real Power Total')
# plt.xlabel('Timesteps, 1 min/unit, showing 4 hours results')

# plt.show()

# =======================================
# modified workflow
# =======================================

In [11]:
# reading configuration settings
configs["window_source_size"] = "180min"
configs["window_target_size"] = "45min"
configs["normalization"] = False

In [12]:
data_wTS

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,DOW_binary_reg_0_min,...,DOW_binary_reg_0_mean,DOW_binary_reg_1_mean,DOW_binary_reg_2_mean,DOW_binary_reg_3_mean,DOW_binary_reg_4_mean,DOW_binary_reg_5_mean,DOW_binary_reg_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,103.650002
2021-12-01 07:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:45:00+00:00,16.614972,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,98.349998
2021-12-01 08:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.493776,0.869589,102.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.000000,0.998135,-1.608123e-16,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.000000,0.997859,-6.540313e-02,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,102.709602
2021-12-08 06:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.000000,0.991445,-1.305262e-01,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.385663,0.922640,105.972198


In [13]:
#################################################################################################################################################
# main function
def main(
    seed, 
    cuda, 
    cell_type, 
    attention_model, 
    la_method, 
    batch_size, 
    hs, 
    save_model, 
    loss_function_qs, 
    normalization,
    configs
):
    
    t0 = time.time()
    np.random.seed(configs["random_seed"])
    torch.manual_seed(configs["random_seed"])
    epochs = configs["num_epochs"]
    window_source_size = configs["window_source_size"]
    window_target_size = configs["window_target_size"]
    resample_interval = configs["resample_interval"]
    window_target_size_count = int(pd.Timedelta(window_target_size) / pd.Timedelta(resample_interval))

    data = data_wTS.astype(np.float32).copy()

    ####################################################################################
    # roll_full_data_s2s
    ####################################################################################
    # setting configuration parameters
    normalization = configs["normalization"]
    window_source_size = configs["window_source_size"]
    window_target_size = configs["window_target_size"]
    resample_interval = configs["resample_interval"]
    target_var = configs["target_var"]

    # normalization
    if normalization:
        print("Transforming data to 0 mean and unit var")
        MU = data.mean(0)  # 0 means take the mean of the column
        data = data - MU
        STD = data.std(0)  # same with std here
        data = data / STD

    # initialize lists
    data_predictor = []
    data_target = []

    tic = time.time()

    # calculate number of rows based on window size in time
    window_source_size_count = int(pd.Timedelta(window_source_size) / pd.Timedelta(resample_interval))
    window_target_size_count = int(pd.Timedelta(window_target_size) / pd.Timedelta(resample_interval))

    # create 3D predictor data 
    data_shifted_predictor = data.iloc[:-window_target_size_count, :]
    for window in data_shifted_predictor.rolling(window=window_source_size):
        if window.shape[0] == window_source_size_count:
            data_predictor.append(
                window.values.reshape((1, window_source_size_count, data_shifted_predictor.shape[1]))
            )

    # create 3D target data
    data_shifted_target = data.loc[data.index>=data.shift(freq=window_source_size).index[0], :][target_var]
    for window in data_shifted_target.rolling(window=window_target_size):
        if window.shape[0] == window_target_size_count:
            data_target.append(
                window.values.reshape((1, window_target_size_count, 1))
            )

    # reshape data dimension
    data_predictor = np.concatenate(np.array(data_predictor), axis=0)
    data_target = np.concatenate(np.array(data_target), axis=0)

    # combine 3D predictor and target data into dictionary
    data = {}
    data['predictor'] = data_predictor
    data['target'] = data_target
    #####################################################################################


    ####################################################################################
    # input_data_split
    ####################################################################################
    configs["sequential_splicer"]["active"] = False

    window_source_size = configs["window_source_size"]
    window_target_size = configs["window_target_size"]
    train_ratio = int(configs["data_split"].split(":")[0]) / 100
    val_ratio = int(configs["data_split"].split(":")[1]) / 100
    test_ratio = int(configs["data_split"].split(":")[2]) / 100

    # If you want to group datasets together into sequential chunks
    # if configs["sequential_splicer"]["active"]:
    #     # Set indices for training set
    #     np.random.seed(seed=configs["random_seed"])
    #     splicer = (
    #         (data.index - data.index[0])
    #         // pd.Timedelta(configs["sequential_splicer"]["window_width"])
    #     ).values
    #     num_chunks = splicer[-1]
    #     num_train_chunks = (train_ratio * num_chunks) - (
    #         (train_ratio * num_chunks) % configs["train_size_factor"]
    #     )
    #     if num_train_chunks == 0:
    #         raise Exception(
    #             "Total number of data chunks is zero. train_size_factor value might be too "
    #             "large compared to the data size. Exiting.."
    #         )

    #     msk = np.zeros(data.shape[0]) + 2
    #     train_chunks = np.random.choice(
    #         np.arange(num_chunks), replace=False, size=int(num_train_chunks)
    #     )
    #     for chunk in train_chunks:
    #         indices = np.where(splicer == chunk)
    #         msk[indices] = 0

    #     # Set indices for validation and test set
    #     remaining_chunks = np.setdiff1d(np.arange(num_chunks), train_chunks)
    #     if test_ratio == 0:
    #         msk[msk != 0] = 1
    #     else:
    #         num_val_chunks = int(
    #             (val_ratio / (1 - train_ratio)) * remaining_chunks.shape[0]
    #         )
    #         val_chunks = np.random.choice(
    #             remaining_chunks, replace=False, size=num_val_chunks
    #         )
    #         for chunk in val_chunks:
    #             indices = np.where(splicer == chunk)
    #             msk[indices] = 1

    # If you DONT want to group data into sequential chunks
    # else:
    # Set indices for training set
    np.random.seed(seed=configs["random_seed"])
    data_size = data['predictor'].shape[0]
    num_ones = (train_ratio * data_size) - (
        (train_ratio * data_size) % configs["train_size_factor"]
    )
    msk = np.zeros(data_size) + 2
    indices = np.random.choice(
        np.arange(data_size), replace=False, size=int(num_ones)
    )
    msk[indices] = 0

    # Set indices for validation and test set
    remaining_indices = np.where(msk != 0)[0]
    if test_ratio == 0:
        msk[remaining_indices] = 1
    else:
        num_val = int(
            (val_ratio / (1 - train_ratio)) * remaining_indices.shape[0]
        )
        val_indices = np.random.choice(
            remaining_indices, replace=False, size=num_val
        )
        msk[val_indices] = 1

    # Assign dataframes
    train_df_predictor = data['predictor'][msk==0, :, :]
    train_df_target = data['target'][msk==0, :, :]
    val_df_predictor = data['predictor'][msk==1, :, :]
    val_df_target = data['target'][msk==1, :, :]
    test_df_predictor = data['predictor'][msk==2, :, :]
    test_df_target = data['target'][msk==2, :, :]

    # # Still save dataframe to file to preserve timeseries index
    # mask = pd.DataFrame()
    # mask["msk"] = msk
    # mask.index = data.index
    # mask.to_hdf(mask_file, key="df", mode="w")

    # Get rid of datetime index
    # train_df_predictor.reset_index(drop=True, inplace=True)
    # train_df_target.reset_index(drop=True, inplace=True)
    # val_df_predictor.reset_index(drop=True, inplace=True)
    # val_df_target.reset_index(drop=True, inplace=True)
    ####################################################################################
    ####################################################################################
        
    # usage_actual = val_df_target[:,0,:].flatten()
    usage_actual = val_df_target.flatten()
    
    print("###############################################")
    print("train_df_predictor.shape = {}".format(train_df_predictor.shape))
    print("val_df_predictor.shape = {}".format(val_df_predictor.shape))
    print("train_df_target.shape = {}".format(train_df_target.shape))
    print("val_df_target.shape = {}".format(val_df_target.shape))
    print("usage_actual.shape = {}".format(usage_actual.shape))
    print("###############################################")
    
    print("Created {} train samples and {} test samples".format(train_df_predictor.shape[0], val_df_predictor.shape[0]))

#################################################################################################################################################
# call the model
    #print("Creating model...")
    INPUT_SIZE = train_df_predictor.shape[-1]
    HIDDEN_SIZE = hs
    CELL_TYPE = cell_type
    LA_METHOD = la_method

    # call the respective model
    if attention_model == 'none':
        model = S2S_Model(CELL_TYPE, INPUT_SIZE, HIDDEN_SIZE, use_cuda=cuda)

    elif attention_model == 'BA':
        model = S2S_BA_Model(CELL_TYPE, INPUT_SIZE, HIDDEN_SIZE, use_cuda=cuda)

    elif attention_model == 'LA':
        model = S2S_LA_Model(CELL_TYPE, LA_METHOD, INPUT_SIZE, HIDDEN_SIZE, use_cuda=cuda)

    if cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        model.cuda()

    print("MODEL ARCHITECTURE IS: ")
    print(model)

    print("\nModel parameters are on cuda: {}".format(next(model.parameters()).is_cuda))

    opt = optim.Adam(model.parameters(), lr=1e-3)
#     loss_fn = nn.MSELoss(reduction='sum')
    loss_fn = quantile_loss
    
    EPOCHES = epochs
    BATCH_SIZE = batch_size

    print("\nStarting training...")

    train_loss = []
    test_loss = []

    i=0
    
    for epoch in range(EPOCHES):
        print("{} ###############################################################".format(i))
        t_one_epoch = time.time()
        print("Epoch {}".format(epoch+1))
        total_usage_loss = 0
        for b_idx in range(0, train_df_predictor.shape[0], BATCH_SIZE):

            x = torch.from_numpy(train_df_predictor[b_idx:b_idx+BATCH_SIZE]).float()
            y_usage = torch.from_numpy(train_df_target[b_idx:b_idx+BATCH_SIZE]).float()

            if cuda:
                x = x.cuda()
                y_usage = y_usage.cuda()

            # encoder forward, for respective models (with and without attention)
            if attention_model == 'none':
                pred_usage, h = model.consume(x)

            elif attention_model == 'BA':
                pred_usage, h, encoder_outputs = model.consume(x)

            elif attention_model == 'LA':
                pred_usage, h, encoder_outputs = model.consume(x)

            # decoder forward, for respective models
            if attention_model == 'none':
                preds = model.predict(pred_usage, h, window_target_size_count)

            elif attention_model == 'BA':
                preds = model.predict(pred_usage, h, encoder_outputs, window_target_size_count)

            elif attention_model == 'LA':
                preds = model.predict(pred_usage, h, encoder_outputs, window_target_size_count)

            # compute lose
            loss_usage = loss_fn(preds, y_usage, qs = loss_function_qs, window_target_size=window_target_size_count)

            # backprop and update
            opt.zero_grad()

            loss_usage.sum().backward()

            opt.step()

            total_usage_loss += loss_usage.item()
            
        train_loss.append(total_usage_loss)
        print("\tTRAINING: {} total train USAGE loss.\n".format(total_usage_loss))

#################################################################################################################################################
# TESTING
        y_usage = None
        pred_usage = None
        preds = None
        total_usage_loss = 0
        all_preds = []

        for b_idx in range(0, val_df_predictor.shape[0], BATCH_SIZE):
            with torch.no_grad():
                x = torch.from_numpy(val_df_predictor[b_idx:b_idx+BATCH_SIZE])
                y_usage = torch.from_numpy(val_df_target[b_idx:b_idx+BATCH_SIZE])

                if cuda:
                    x = x.cuda()
                    y_usage = y_usage.cuda()

                # encoder forward, for respective models
                if attention_model == 'none':
                    pred_usage, h = model.consume(x)

                elif attention_model == 'BA':
                    pred_usage, h, encoder_outputs = model.consume(x)

                elif attention_model == 'LA':
                    pred_usage, h, encoder_outputs = model.consume(x)

                # decoder forward, for respective models
                if attention_model == 'none':
                    preds = model.predict(pred_usage, h, window_target_size_count)

                elif attention_model == 'BA':
                    preds = model.predict(pred_usage, h, encoder_outputs, window_target_size_count)

                elif attention_model == 'LA':
                    preds = model.predict(pred_usage, h, encoder_outputs, window_target_size_count)

                # compute loss
                loss_usage = loss_fn(preds, y_usage, qs = loss_function_qs, window_target_size=window_target_size_count)

                if (epoch == epochs-1):
                    all_preds.append(preds)
                    
                total_usage_loss += loss_usage.item()

        test_loss.append(total_usage_loss)

        print("\tTESTING: {} total test USAGE loss".format(total_usage_loss))
        print("\tTESTING:\n")
        print("\tSample of prediction:")
        print("\t\t TARGET: {}".format(y_usage[-1].cpu().detach().numpy().flatten()))
        print("\t\t   PRED: {}\n\n".format(preds[-1].cpu().detach().numpy().flatten()))

        y_last_usage = y_usage[-1].cpu().detach().numpy().flatten()
        pred_last_usage = preds[-1].cpu().detach().numpy().flatten()
        t2_one_epoch = time.time()
        time_one_epoch = t2_one_epoch - t_one_epoch
        print("TIME OF ONE EPOCH: {} seconds and {} minutes".format(time_one_epoch, time_one_epoch/60.0))
        
        i+=1

####################################################################################################
# SAVING MODEL
    if save_model:
        torch.save(model.state_dict(), "MODEL_w:__seed={}_cell_type={}_attention_model={}_la_method={}_T={}_N={}_bs={}_hs={}".format(
            seed, cell_type, attention_model, la_method,
            window_source_size, window_target_size, batch_size, hs))

#################################################################################################################################################
# RESULTS
    # for plotting and accuracy
    preds = torch.cat(all_preds, 0)
    preds = preds.cpu().detach().numpy().flatten()
    actual = val_df_target.flatten()

    # for loss plotting
    train_loss_array = np.asarray(train_loss)
    test_loss_array = np.asarray(test_loss)
    len_loss = np.arange(len(train_loss_array))

    # unnormalizing 1
    if normalization:
        preds_unnorm = (preds*std_usage) + mu_usage
    else:
        preds_unnorm = preds
        
    print("###############################################")
    print("preds_unnorm.shape = {}".format(preds_unnorm.shape))
    print("usage_actual.shape = {}".format(usage_actual.shape))
    print("###############################################")

    # using the actual usage from top of script here
    mae3 = (sum(abs(usage_actual - preds_unnorm)))/(len(usage_actual))
    mape3 = (sum(abs((usage_actual - preds_unnorm)/usage_actual)))/(len(usage_actual))

    # for std
    mape_s = (abs((usage_actual - preds_unnorm)/usage_actual))
    s = mape_s.std()
    mae_s = abs(usage_actual - preds_unnorm)
    s2 = mae_s.std()
    print("\n\tACTUAL ACC. RESULTS: MAE, MAPE: {} and {}%".format(mae3, mape3*100.0))
    
    if not os.path.exists(f'results/{site}'):
        os.makedirs(f'results/{site}')
    
    pd.DataFrame(preds, columns = [f'q{loss_function_qs}']).to_csv(f'results/{site}/q{loss_function_qs}.csv', index = None)
    pd.DataFrame(actual, columns = [f'actual']).to_csv(f'results/{site}/actual.csv', index = None)

    # total time of run
    t1 = time.time()
    total = t1-t0
    print("\nTIME ELAPSED: {} seconds OR {} minutes".format(total, total/60.0))
    print("\nEnd of run")
    plt.show()
    for_plotting = [usage_actual, preds_unnorm, y_last_usage, pred_last_usage]
    
    return s, s2, mape_s, mae_s, mae3, mape3, total/60.0, train_loss, test_loss, for_plotting

In [24]:
# cafe data is too large to set a high epochs number to run on my local machine

# for loss_function_qs in [0.05, 0.25, 0.50, 0.75, 0.95]: #[0.50]:#[0.05, 0.50, 0.95]:
#     print(f"Processing loss function alpha: {loss_function_qs}")
#     if __name__ == "__main__":
#         s, s2, mape_s, mae_s, mae, mape, total_mins, train_loss, test_loss, for_plotting = main(
#             seed=42, 
#             cuda=False,
#             cell_type='lstm', 
#             attention_model='BA', 
#             la_method='none',
#             batch_size=256, 
#             hs=64, 
#             save_model=False, 
#             loss_function_qs = loss_function_qs, 
#             normalization = False,
#             configs = configs
#         )
        

import plotly.graph_objects as go
from plotly.colors import n_colors

#################################################
# Common settings for graphics
#################################################
font_family = "Raleway, PT Sans Narrow, Overpass, Arial"
width_graphic = 1500
color_base = "rgb(236,200,178)"
color_accent = "rgb(80,70,69)"
c_target = "rgb(228,26,28)"
c_list = n_colors(color_base, color_accent, 3, colortype="rgb")

fig = go.Figure()

i_qntl = 1
for qntl in [["0.05","0.95"], ["0.25","0.75"], ["0.05"]]:
    
    if len(qntl) == 2:

        df_low = pd.read_csv(f'results/{site}/q{qntl[0]}.csv')
        df_high = pd.read_csv(f'results/{site}/q{qntl[1]}.csv')

        fig.add_trace(
            go.Scatter(
                y=df_low.iloc[:,0],
                mode="lines",
#                 name="Quantile{}".format(i_qntl),
#                 legendgroup="Quantile{}".format(i_qntl),
#                 showlegend=True,
#                 xaxis="x",
#                 opacity=0.1,
#                 line=dict(color=c_list[i_qntl], width=0),
            ),
        )

        fig.add_trace(
            go.Scatter(
                y=df_high.iloc[:,0],
                mode="lines",
#                 name="Quantile{}".format(i_qntl),
#                 legendgroup="Quantile{}".format(i_qntl),
#                 showlegend=False,
#                 xaxis="x",
#                 fill="tonexty",
#                 opacity=0.1,
#                 line=dict(color=c_list[i_qntl], width=0),
            ),
        )
    
    i_qntl+=1
    
df_actual = pd.read_csv(f'results/{site}/actual.csv')

fig.add_trace(
    go.Scatter(
        y=df_actual.iloc[:,0],
        mode="lines",
        name="Target",
        legendgroup="Target",
        line=dict(color=c_target, width=1),
        xaxis="x",
    ),
)

fig.update_layout(
    width=700,
    height=300,
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=0,
    ),
    plot_bgcolor="rgb(245,245,245)",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5,
        font=dict(
            size=12,
            family=font_family,
        ),
    ),
)
    
fig.show()

In [21]:
df_high.iloc[:,0]

0      20.264528
1      29.095420
2      32.752693
3      20.271590
4      29.087760
         ...    
196    29.768808
197    33.264610
198    20.156776
199    29.073414
200    32.611244
Name: q0.75, Length: 201, dtype: float64